In [1]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# updating nipype from 1.7.0 -> 1.8.1, will hopefully ifx no PE estimates founds error # didn't fix

221003-10:10:23,276 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

In [2]:
dataset = 'Leipzig_7T_SM'
# dataset = 'Leipzig_7T_GdH'

#### !!! must manually change subject numbers in nipype code based on dataset !!!

In [3]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

if dataset == 'Leipzig_7T_SM':
    task = 'stop'
    spaces = ['T1w'] 
#     model_ns = ['0']
    model_ns = ['0']
    fwhms = ['5p0']#['3p125']#['1p6']
    
elif dataset == 'Leipzig_7T_GdH':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['1p5']
    
if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model

subject_ids = [x.split('/')[-5].split('-')[-1] for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 
                                     'subject_level_model', dataset, 'sub-*', 'func',f'fwhm-{fwhms[0]}',f'model-{model_ns[0]}',
                                     f'*task-{task}_space-MNI*contrast-0_desc-zstat*')))]
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', dataset)
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', dataset)

template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "group_level_model", dataset,f'model-{model_ns[0]}')

templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stop_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stop_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stop_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}


In [4]:
print(f"""
dataset : {dataset}
fwhm : {fwhms}
model_n : {model_ns}
task : {task}
no. subs : {len(subject_ids)}
subs : {subject_ids}
work_dir : {work_dir}
slm folder : {slm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level2_cope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level2_varcope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level2_tdof'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
""")


dataset : Leipzig_7T_SM
fwhm : ['5p0']
model_n : ['0']
task : stop
no. subs : 15
subs : ['01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '13', '15', '16', '17', '18']
work_dir : /home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM
slm folder : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/subject_level_model/Leipzig_7T_SM
output dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0
contrasts : ['0', '1', '2', '3', '4', '5']

## if uneven number of template / designs / subs the code will crash
no.copes = 15
no.varvopes = 15
no.tdofs = 15



In [5]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [6]:
##
workflow = pe.Workflow(name='feat_level3_sst_hp')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [7]:
def get_subject_ids():
#     if dataset == 'Leipzig_7T_SM':
    subject_ids = ['01', '02', '03', '05', '06', '07', '08', '09', '10', '11', '13', '15', '16', '17', '18']
        
#     elif dataset == 'Leipzig_7T_GdH':
#     subject_ids = ['BI3T', 'DA9T', 'FMFT', 'GAIT', 'HCBT', 'KCAT', 'KP6T', 'LV2T', 'MRCT', 'NM3T', 'PF5T', 
#                        'RSIT', 'SC1T', 'SPGT', 'TS6T', 'UM2T', 'VL1T', 'WSFT', 'WW2T', 'ZK4T']
        
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids): #=subject_ids):
    print(second_level_contrast)
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors


contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')



In [8]:
# flameo = pe.Node(
#     interface=fsl.FLAMEO(run_mode='flame1'),
#     name="flame1")

# flameo.inputs.mask_file = template_brain_mask

# workflow.connect([
#     (copemerge, flameo, [('merged_file', 'cope_file')]),
#     (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
#     (level3model, flameo, [('design_mat', 'design_file'),
#                            ('design_con', 't_con_file'), 
#                            ('design_grp', 'cov_split_file')]),
# ])

flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [9]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [10]:
# ## Datasink
# datasink = pe.Node(nio.DataSink(), name='sinker')
# datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-sstmsit","task-sst")

# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

# ## todo: substitutions
# workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
# workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
# workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
# workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

# ## cluster results
# workflow.connect(grf_cluster, 'threshold_file', datasink, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'grf_localmax_txt_file')
# workflow.connect(grf_cluster, 'index_file', datasink, 'cluster_indices')

In [11]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir

if len(fwhms[0]) == 3:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]
elif len(fwhms[0]) == 5:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]    

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [12]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 100})

221003-10:10:39,559 nipype.workflow INFO:
	 Workflow feat_level3_sst_hp settings: ['check', 'execution', 'logging', 'monitoring']
221003-10:10:39,913 nipype.workflow INFO:
	 Running in parallel.
221003-10:10:40,258 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 100.00/100.00, Free processors: 10/10.
221003-10:10:40,358 nipype.workflow INFO:
	 [Job 0] Cached (feat_level3_sst_hp.subject_id_getter).
221003-10:10:40,360 nipype.workflow INFO:
	 [Job 1] Cached (feat_level3_sst_hp.get_volume).
221003-10:10:40,494 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_1_fwhm_5p0_model_n_0/selector".
221003-10:10:40,493 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contras

221003-10:10:42,558 nipype.workflow INFO:
	 [Node] Finished "contrastgen_l3", elapsed time 0.031852s.
221003-10:10:44,261 nipype.workflow INFO:
	 [Job 8] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:44,263 nipype.workflow INFO:
	 [Job 9] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:44,264 nipype.workflow INFO:
	 [Job 10] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:44,269 nipype.workflow INFO:
	 [Job 11] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:44,270 nipype.workflow INFO:
	 [Job 12] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:44,272 nipype.workflow INFO:
	 [Job 13] Completed (feat_level3_sst_hp.contrastgen_l3).
221003-10:10:44,274 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 14 jobs ready. Free memory (GB): 99.20/100.00, Free processors: 6/10.
                     Currently running:
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
  

221003-10:11:08,392 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.l3model" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_3_fwhm_5p0_model_n_0/l3model".
221003-10:11:08,393 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.l3model" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_4_fwhm_5p0_model_n_0/l3model".
221003-10:11:08,478 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
221003-10:11:08,479 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
221003-10:11:08,480 nipype.workflow INFO:
	 [Node] Executing "varcopemerge" <nipype.interfaces.fsl.utils.Merge>
221003-10:11:08,489 nipype.workflow INFO:
	 [Node] Executing "l3model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
221003-10:11:08,478 n

221003-10:11:29,453 nipype.workflow INFO:
	 [Node] Finished "varcopemerge", elapsed time 20.95916s.
221003-10:11:30,305 nipype.workflow INFO:
	 [Job 25] Completed (feat_level3_sst_hp.varcopemerge).
221003-10:11:30,308 nipype.workflow INFO:
	 [MultiProc] Running 9 tasks, and 3 jobs ready. Free memory (GB): 98.20/100.00, Free processors: 1/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
221003-10:11:30,414 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.flameo" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_wo

221003-10:15:40,753 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate0" - collecting precomputed outputs
221003-10:15:40,773 nipype.workflow INFO:
	 [Node] "_smoothestimate0" found cached.
221003-10:15:40,793 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame1/smoothestimate/mapflow/_smoothestimate1".
221003-10:15:40,809 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate1" - collecting precomputed outputs
221003-10:15:40,825 nipype.workflow INFO:
	 [Node] "_smoothestimate1" found cached.
221003-10:15:40,845 nipype.workflow INFO:
	 [Node] Executing "_smoothestimate0" <nipype.interfaces.fsl.model.SmoothEstimate>
221003-10:15:42,141 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate0", elapsed time 1.282443s.
221003-10:15:42,597 nipype.workflow INFO:
	 [Job 44] Completed (feat_level3_sst_hp.smoothestima

...

221003-15:48:50,77 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_1_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_3.1/_grf_cluster0/zstat1_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-1_grouplevelcontrast-1_flame-12_desc-zstat_voxelthreshold-3.1.nii.gz
221003-15:48:50,78 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_1_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_3.1/_grf_cluster1/zstat2_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/model-0/model-0_fwhm-5p0_subjectlevel

221003-15:50:44,147 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster1" - collecting precomputed outputs
221003-15:50:44,147 nipype.workflow INFO:
	 [Node] "_grf_cluster1" found cached.
221003-15:50:44,255 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 2.112515s.
221003-15:50:46,92 nipype.workflow INFO:
	 [Job 168] Completed (_grf_cluster0).
221003-15:50:46,93 nipype.workflow INFO:
	 [Job 66] Completed (feat_level3_sst_hp.grf_cluster).
221003-15:50:46,96 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 2 jobs ready. Free memory (GB): 99.80/100.00, Free processors: 9/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
221003-15:50:46,146 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.sinker" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_2.3/sinker".
221003-15:50:46,151

221003-15:50:48,194 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/third_level_model/level3_zstats/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/zstat2.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-2_flame-12_desc-zstat.nii.gz
221003-15:50:48,194 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/third_level_model/level3_copes/_contrast_n_2_fwhm_5p0_model_n_0/_run_mode_flame12/cope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-2_grouplevelcontrast-1_flame-12_desc-cope.nii.gz
221003-15:50:48,195 nipype.interface INFO:
	 sub: /home/scotti/proj

221003-15:53:00,262 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_2.3/grf_cluster/mapflow/_grf_cluster1".
221003-15:53:00,263 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_3.1/grf_cluster/mapflow/_grf_cluster0".
221003-15:53:00,263 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/Leipzig_7T_SM/feat_level3_sst_hp/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/_threshold_3.1/grf_cluster/mapflow/_grf_cluster1".
221003-15:53:00,266 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>


221003-15:53:04,318 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/third_level_model/level3_varcopes/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/varcope1.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-0_grouplevelcontrast-1_flame-12_desc-varcope.nii.gz
221003-15:53:04,319 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/third_level_model/level3_copes/_contrast_n_0_fwhm_5p0_model_n_0/_run_mode_flame12/cope2.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/Leipzig_7T_SM/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-0_grouplevelcontrast-2_flame-12_desc-cope.nii.gz
221003-15:53:04,319 nipype.interface INFO:
	 sub: /home/scott

In [25]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20220912-155838-scotti-flameo.bI.b1.c2-2ec71381-2a71-4012-b9e7-080e40b5834e.pklz')
res

{'node': feat_level3_sst_hp.flameo.bI.b1.c2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node flameo.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in ru

In [13]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/scotti/projects/3t_7t_sst_comparison/processing/crashdumps/crash-20220909-213138-scotti-flameo.bI.b0.c2-a68890de-6d23-44cb-a27b-2155bc31ffa6.pklz')
res

{'node': feat_level3_sst_hp.flameo.bI.b0.c2,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 521, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 639, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 750, in _run_command\n    raise NodeExecutionError(\n',
  'nipype.pipeline.engine.nodes.NodeExecutionError: Exception raised while executing Node flameo.\n\nTraceback (most recent call last):\n  File "/home/scotti/.local/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 401, in ru